In [ ]:
import time
import os
import numpy as np
import torch
from torch.utils.data import random_split
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from google.colab import drive
import os
import torch.nn as nn
drive.mount('/content/gdrive')
torch.manual_seed(0)

In [ ]:

path = "/content/gdrive/MyDrive/APS360/360Project/final_dataset"
train_dir = os.path.join(path, 'train/')
val_dir = os.path.join(path, 'val/')
test_dir = os.path.join(path, 'test/')
classes =['Autos & Vehicles','Food & Drink','Pets & Animals','Science & Education','Sports']

data_transform = transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandAugment(),transforms.ToTensor()])
train_set = datasets.ImageFolder(train_dir, transform=data_transform)

data_transform = transforms.Compose([transforms.RandomResizedCrop(224),transforms.ToTensor()])
validation_set = datasets.ImageFolder(val_dir, transform=data_transform)
test_set = datasets.ImageFolder(test_dir, transform=data_transform)

In [ ]:
# define dataloader parameters
batch_size  = 500
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size,num_workers=num_workers, shuffle=True)


In [ ]:
#Model CNN 
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.conv3 = nn.Conv2d(10, 20, 4)
        self.conv4 = nn.Conv2d(20, 40, 4)
        self.fc1 = nn.Linear(4840, 320)
        self.fc2 = nn.Linear(320, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 4840)
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        return x


In [ ]:
#GoogleENET
googleNet = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

In [ ]:
#Artifical Neural Network Architecture (GoogleNet)
Google = googleNet
class ANNClassifier_GOOGLE(nn.Module):
    def __init__(self):
        super(ANNClassifier_GOOGLE, self).__init__()
        self.fc1 = nn.Sequential(nn.Dropout(0.5),nn.Linear(1000,6))
        #self.fc2 = nn.Linear(10, 6)
    def forward(self, x):
        x = x.view(-1, 1000) #flatten feature data
        x = F.relu(self.fc1(x))
        #x = self.fc2(x)
        return x

Helper function

In [ ]:
def get_accuracy(model, train=False):
    model.eval()
    with torch.no_grad():
      if train:
          data_loader = train_loader
      else:
          data_loader = val_loader

      correct = 0
      total = 0
      for imgs, labels in data_loader:
        
         
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        #改这里 change here to switch model
        output = model(Google(imgs))
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

Train code


In [ ]:
def train(model, data, lr,batch_size, num_epochs=1,train_loader=train_loader):
    #train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    # train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           #num_workers=num_workers, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    start_time=time.time()
    for epoch in range(num_epochs):
        model.train()
        mini_b=0
        mini_batch_correct = 0
        Mini_batch_total = 0
        for imgs, labels in iter(train_loader):
          
            
            #############################################
            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            #############################################

          #### ALNC is alexNet.features (AlexNet without classifier) ####
          #### VGG is vggNet.fearures (VggNet without classifier) ####
          #### RES is resNet
          #改这里 change here to switch model
            out = model(Google(imgs))             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch



            ##### Mini_batch Accuracy ##### We don't compute accuracy on the whole training set in every iteration!
            pred = out.max(1, keepdim=True)[1]
            mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
            Mini_batch_total = imgs.shape[0]
            train_acc.append((mini_batch_correct / Mini_batch_total))
           ###########################

          # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            val_acc.append(get_accuracy(model, train=False))  # compute validation accuracy
            n += 1
            mini_b += 1
            print("Iteration: ",n,'Progress: % 6.2f ' % ((epoch * len(train_loader) + mini_b) / (num_epochs * len(train_loader))*100),'%', "Time Elapsed: % 6.2f s " % (time.time()-start_time))


        print ("Epoch %d Finished. " % epoch ,"Time per Epoch: % 6.2f s "% ((time.time()-start_time) / (epoch +1)))
        model_path = get_model_name("google_drop_out_full_last", batch_size=batch_size, learning_rate=lr, epoch=epoch)
        torch.save(model.state_dict(), model_path)

    end_time= time.time()
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Training")
    plt.plot(iters, val_acc, label="Validation")    
    plt.xlabel("Iterations")
    plt.ylabel("Validation Accuracy")
    plt.legend(loc='best')
    plt.show()

    train_acc.append(get_accuracy(model, train=True))
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    print ("Total time:  % 6.2f s  Time per Epoch: % 6.2f s " % ( (end_time-start_time), ((end_time-start_time) / num_epochs) ))
    

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "/content/gdrive/MyDrive/APS360/360Project/Models/model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [ ]:
use_cuda = True
model = ANNClassifier_GOOGLE()
if use_cuda and torch.cuda.is_available():
  model.cuda()
  Google.cuda()
  print(torch.cuda.get_device_name(0))
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
  
#proper model
train(model, [],lr=0.001, batch_size=batch_size, num_epochs=30, train_loader=train_loader)

In [ ]:
model_path = get_model_name("google_drop_out_full", batch_size=batch_size, learning_rate=0.001, epoch=1)
torch.save(model.state_dict(), model_path)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=50,num_workers=0, shuffle=True)

def get_test_accuracy(model):
    data_loader = test_loader
    correct = 0
    total = 0
    predict=[]
    truth=[]
    for imgs, labels in data_loader:
        
         
        #############################################
        #To Enable GPU Usage
        #if torch.cuda.is_available():
          #imgs = imgs.cuda()
          #labels = labels.cuda()
        #############################################
        #改这里 change here to switch model
        output = model(Google(imgs))
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]

        predict.append(pred)
        truth.append(labels)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    #return predict,truth
    return correct / total

In [ ]:
model = ANNClassifier_GOOGLE()
model_path = get_model_name("google_drop_out_full", batch_size=400, learning_rate=0.001, epoch=10)
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

In [ ]:
#model.to('cpu')
model.eval()
get_test_accuracy(model)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
def draw_confussion_matrix(model,data_loader):
  model.eval()
  predict=[]
  truth=[]
  real_predict=[]
  real_symbols=[]
  for imgs, labels in data_loader:
    output = model(Google(imgs))
    pred = output.max(1, keepdim=True)[1]
    predict.append(pred)
    truth.append(labels)
  for predict_batch in predict:
    for predicts in predict_batch:
      real_predict.append(classes[predicts.item()])
  for symbol_batch in truth:
    for symbol in symbol_batch:
      real_symbols.append(classes[symbol.item()])
  print(len(real_predict),len(real_symbols))
  df_cm = pd.DataFrame(confusion_matrix(real_symbols,real_predict), index = [i for i in classes],
                  columns = [i for i in classes])
  plt.figure(figsize = (10,7))
  sn.heatmap(df_cm, annot=True) #cite:https://seaborn.pydata.org/generated/seaborn.heatmap.html


In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=50,num_workers=0, shuffle=True)
draw_confussion_matrix(model,test_loader)